<a href="https://colab.research.google.com/github/hhnafis/PIAIC_LangChain-Function-Tool-Calling-Calculator/blob/main/PIAIC_LangChain_Function_Tool_Calling_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install dependencies
!pip install -Uq langchain langchain-google-genai python-dotenv langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00


In [2]:
#Loading API key
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

In [3]:
#Defining calculator tool with optional enhancements
import math

class Calculator:
    def calculate(self, expression: str) -> str:
      try:
        if any(func in expression.lower() for func in ['log', 'sin', 'cos', 'tan', 'sqrt']):

          #Handle logrithmic problems
          if 'log' in expression.lower():
            try:
              base,num = map(float(expression.lower().replace('log', '').split('')))
              result = math.log(num, base)
              return str(result)
            except:
              try:
                num = float(expression.lower().replace('log','').split(''))
                result = math.log10(num)
                return str(result)
              except ValueError as e:
                return f"Error: {e}. Invalid logarithm input"

            #Handle trignometery problems
          elif 'sin' in expression.lower():
            angle = float(expression.lower().replace('sin','').split(''))
            result = math.sin(math.radians(angle))
            return str(result)
          elif 'cos' in expression.lower():
            angle = float(expression.lower().replace('cos','').split(''))
            result = math.cos(math.radians(angle))
            return str(result)
          elif 'tan' in expression.lower():
            angle = float(expression.lower().replace('tan','').split(''))
            result = math.tan(math.radians(angle))
            return str(result)
          elif 'sqrt' in expression.lower():
            num = float(expression.lower().replace('sqrt','').split(''))
            result = math.sqrt(num)
            return str(result)
          else:
            return "Invalid trigonometric function"
        else:

          #Handle simple arithmatic problems found in the expression
          result = eval(expression, {'__builtins__':None},{})
          return str(result)
      except Exception as e:
        return f"Error: {e}"




In [4]:
#Created the Tool Wrapper for LangChain
from langchain_core.tools import tool

@tool
def calculator(expression:str) -> str:
  """
  Perform mathematical calculations.
  Input: A mathematical expression as a string (e.g., "2 + 2")
  Output: Result of the calculation as a string. """

  calc = Calculator()
  return calc.calculate(expression)

In [5]:
#Seting Up the Google Gemini Flash Model
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GOOGLE_GEMINI_API_KEY= userdata.get('GOOGLE_API_KEY')

# Initializing the Gemini model
gemini_model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash",api_key=GOOGLE_GEMINI_API_KEY)


In [6]:
# Initializing LLM with tool-calling capabilities.

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import AgentExecutor
tools = [calculator]

# Initializing the agent with tool-calling capabilities
agent = initialize_agent(
    tools, gemini_model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

<ipython-input-6-2570a2cdc807>:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
!pip install -Uq langchain --upgrade #Upgraded langchain to the newest version
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# Seting up memory to maintain conversation context
memory = ConversationBufferMemory()

# Defining a prompt template that incorporates the conversation history
template = """You are a helpful assistant having a conversation with a user.
Current Conversation:
{history}
Human: {human_input}
Assistant:"""
prompt = PromptTemplate(
    input_variables=["history", "human_input"], template=template
)

# Build ingthe LLMChain
llm_chain = LLMChain(llm=gemini_model, prompt=prompt, memory=memory)

# Function to interact with the chain and agent
def conversational_agent(user_input):
  # Get LLM response considering conversation history
  llm_response = llm_chain.run(human_input=user_input)

  # If the response suggests using a tool, call the agent
  if "calculator" in llm_response.lower():
    agent_response = executor.invoke(llm_response)  # Use executor for tool calls
    return agent_response
  else:
    return llm_response



In [10]:
#Simple queries
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", conversational_agent(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?
Response: 25 multiplied by 4 is 100.
----------------------------------------
Query: Now divide the result by 5.
Response: 100 divided by 5 is 20.
----------------------------------------
Query: Add 10 to that.
Response: Adding 10 to 20 gives you 30.
----------------------------------------


In [11]:
#Optional enhancement

queries = [
    "Calculate the log of 100 to the base 10.",
    "What is the square root of 64?",
    "Find the tangent of 45 degrees.",
    "Find the cosine of 30 degrees."
]

for q in queries:
    print("Query:", q)
    print("Response:", conversational_agent(q))
    print("-" * 40)

Query: Calculate the log of 100 to the base 10.
Response: The logarithm of 100 to the base 10 is 2.  (Because 10 to the power of 2 equals 100).
----------------------------------------
Query: What is the square root of 64?
Response: The square root of 64 is 8.
----------------------------------------
Query: Find the tangent of 45 degrees.
Response: The tangent of 45 degrees is 1.
----------------------------------------
Query: Find the cosine of 30 degrees.
Response: The cosine of 30 degrees is √3/2  or approximately 0.866.
----------------------------------------


In [13]:
print(conversational_agent("What are the calculations, I asked uptill here?"))

Here's a summary of the calculations we performed:

1. **Multiplication:** 25 * 4 = 100
2. **Division:** 100 / 5 = 20
3. **Addition:** 20 + 10 = 30
4. **Logarithm:** log₁₀(100) = 2
5. **Square Root:** √64 = 8
6. **Trigonometry:** tan(45°) = 1
7. **Trigonometry:** cos(30°) = √3/2 ≈ 0.866


#End